ABCD 중첩임. 두 개의 블록에 관측치가 8개씩 들어 있다.

실험 결과:

|처리조합|I|처리조합|I|
|---|---|---|---|
|$(1)$|$90$|$d$|$98$|
|$a$|$74$|$ad$|$72$|
|$b$|$81$|$bd$|$87$|
|$ab$|$83$|$abd$|$85$|
|$c$|$77$|$cd$|$99$|
|$ac$|$81$|$acd$|$79$|
|$bc$|$88$|$bcd$|$87$|
|$abc$|$73$|$abcd$|$80$|

In [11]:
import pandas as pd
import statsmodels.api as sm

temp = pd.DataFrame({
    'Treament': ['(1)', 'a','b','ab','c','ac','bc','abc','d','ad','bd','abd','cd','acd','bcd','abcd'],
})

temp['a'] = temp['Treament'].map(lambda x: 1 if 'a' in x else -1)
temp['b'] = temp['Treament'].map(lambda x: 1 if 'b' in x else -1)
temp['c'] = temp['Treament'].map(lambda x: 1 if 'c' in x else -1)
temp['d'] = temp['Treament'].map(lambda x: 1 if 'd' in x else -1)
temp

,Treament,a,b,c,d
0,(1),-1,-1,-1,-1
1,a,1,-1,-1,-1
2,b,-1,1,-1,-1
3,ab,1,1,-1,-1
4,c,-1,-1,1,-1
5,ac,1,-1,1,-1
6,bc,-1,1,1,-1
7,abc,1,1,1,-1
8,d,-1,-1,-1,1
9,ad,1,-1,-1,1


In [12]:
temp['ABCD_Block'] = temp[['a','b','c','d']].prod(1)
temp['Y'] = [90,74,81,83,77,81,88,73,98,72,87,85,99,79,87,80]
temp

,Treament,a,b,c,d,ABCD_Block,Y
0,(1),-1,-1,-1,-1,1,90
1,a,1,-1,-1,-1,-1,74
2,b,-1,1,-1,-1,-1,81
3,ab,1,1,-1,-1,1,83
4,c,-1,-1,1,-1,-1,77
5,ac,1,-1,1,-1,1,81
6,bc,-1,1,1,-1,1,88
7,abc,1,1,1,-1,-1,73
8,d,-1,-1,-1,1,-1,98
9,ad,1,-1,-1,1,1,72


# 효과 추정

In [ ]:
from DOE import effect_est

for fac in temp['Treament']:
    if fac == '(1)': continue
    if fac == 'abcd': 
        print(f'Effect of {fac.upper()}Block:',effect_est(data=temp, sign_col=[f for f in fac], value_cols=['Y']))
        break
    print(f'Effect of {fac.upper()}:',effect_est(data=temp, sign_col=[f for f in fac], value_cols=['Y']))

Effect of A: -10.0
Effect of B: -0.75
Effect of AB: 4.5
Effect of C: -0.75
Effect of AC: 0.5
Effect of BC: -1.25
Effect of ABC: -6.0
Effect of D: 5.0
Effect of AD: -3.75
Effect of BD: -1.5
Effect of ABD: 4.75
Effect of CD: 1.5
Effect of ACD: -0.25
Effect of BCD: -2.0
Effect of ABCDBlock: 3.25


# 분산분석

In [25]:
model = sm.OLS.from_formula(formula="Y ~ a+b+c+d+a:b+a:c+a:d+b:c+b:d+a:b:c+a:b:d+ABCD_Block", data=temp).fit()
model.summary()

c:\Users\USER\miniconda3\envs\doe_env\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     9.252
Date:                Wed, 04 Dec 2024   Prob (F-statistic):             0.0463
Time:                        15:54:14   Log-Likelihood:                -26.353
No. Observations:                  16   AIC:                             78.71
Df Residuals:                       3   BIC:                             88.75
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     83.3750      0.725    114.954      0.000      81.067      85.683
a             -5.0000      0.725     -6.894      0.006      -7.308      -2.692
b             -0.3750      0.725     -0.517      0.641      -2.683       1.933
c             -0.3750      0.725     -0.517      0.641      -2.683       1.933
d              2.5000      0.725      3.447      0.041       0.192       4.808
a:b            2.2500      0.725      3.102      0.053      -0.058       4.558
a:c            0.2500      0.725      0.345      0.753      -2.058       2.558
a:d           -1.8750      0.725     -2.585      0.081      -4.183       0.433
b:c           -0.6250      0.725     -0.862      0.452      -2.933       1.683
b:d           -0.7500      0.725     -1.034      0.377      -3.058       1.558
a:b:c         -3.0000      0.725     -4.136      0.026      -5.308      -0.692
a:b:d          2.3750      0.725      3.275      0.047       0.067       4.683
ABCD_Block     1.6250      0.725      2.240      0.111      -0.683       3.933
==============================================================================
Omnibus:                        1.094   Durbin-Watson:                   0.884
Prob(Omnibus):                  0.579   Jarque-Bera (JB):                0.745
Skew:                          -0.000   Prob(JB):                        0.689
Kurtosis:                       1.943   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
sm.stats.anova_lm(model, typ=3).round(4)

,sum_sq,df,F,PR(>F)
Intercept,111222.25,1.0,13214.5248,0.0000
a,400.00,1.0,47.5248,0.0063
b,2.25,1.0,0.2673,0.6408
c,2.25,1.0,0.2673,0.6408
d,100.00,1.0,11.8812,0.0410
a:b,81.00,1.0,9.6238,0.0532
a:c,1.00,1.0,0.1188,0.7531
a:d,56.25,1.0,6.6832,0.0814
b:c,6.25,1.0,0.7426,0.4522
b:d,9.00,1.0,1.0693,0.3772
